In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
import tensorflow as tf
from keras.layers import Dense , Activation , Dropout, Embedding, SpatialDropout1D, LSTM , TextVectorization, TimeDistributed
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

In [3]:
from bs4 import BeautifulSoup
import os
import re

In [4]:
directory = 'Corpus\\corpus_morphological_analysis'
file_paths = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    file_paths.append(f)    

In [5]:
temp_file_paths = file_paths[:90]

In [6]:
content = []
#i = 0
for filepath in temp_file_paths :
    #print(i)
    with open(filepath, encoding='utf-8') as f :
        html = f.read()
    soup = BeautifulSoup(html, features="html.parser")
    for script in soup(["script", "style"]):
        script.extract()  
    text = soup.get_text()
    content.append(text)
    #i+=1

In [7]:
len(content)

90

In [8]:
split_list = []
for item in content : 
    tmp = item.splitlines()
    split_list.append(tmp)

In [9]:
work_list = []
for k in split_list :
    l = [item for item in k if 'لا توجد نتائج لتحليل هذه الكلمة' not in item]
    tmp_l = [item.replace("#",'') for item in l]
    work_list.append(tmp_l)

In [10]:
final_list = []
for k in work_list :
    tst = [item.split(':') for item in k]
    final_list.append(tst)

In [11]:
# identifie le prefixe, racine et le suffixe et les placent dans un dictionnaire
def identify(word_l):
    dictt = {}
    dictt['word'] = word_l[1]
    # le cas s'il existe un préfixe
    if word_l[2] != '' and word_l[2] != ' ' :   
        if word_l[4] not in word_l[0]: 
            if word_l[5] in word_l[0] and word_l[5] != '': 
                dictt['prefixe'] = word_l[2]
                dictt['root'] = word_l[8]
                dictt['suffixe'] = word_l[9]
            elif word_l[3] in word_l[0] and word_l[3] != '':
                dictt['prefixe'] = word_l[2]
                dictt['root'] = word_l[3]
                dictt['suffixe'] = ''
        else :
            dictt['prefixe'] = word_l[2]
            dictt['root'] = word_l[7]
            dictt['suffixe'] = word_l[8]
    # s'il n'existe pas un préfixe
    else : 
        if word_l[2] == '' : 
            dictt['prefixe'] = word_l[2]
            dictt['root'] = word_l[6]
            dictt['suffixe'] = word_l[7]
        elif  word_l[2] == ' ' :
            dictt['prefixe'] = ''
            dictt['root'] = word_l[3]
            dictt['suffixe'] = ''    
    return dictt

In [12]:
# filtre la liste de mots en liste de dictionnaires
def word_to_dict_list(wordlist):
    dictlist = []
    for k in wordlist : 
        dictlist.append(identify(k))
    return dictlist

In [13]:
final = []
for k in final_list: 
    for j in k :
        final.append(identify(j))

In [14]:
print(final[1000])

{'word': 'الْجِمْعَةَ', 'prefixe': 'ال', 'root': 'جمع', 'suffixe': 'ة'}


In [15]:
def dic_to_list(listt):
    L = []
    for k in listt : 
        tmp = []
        #print(k)
        if len(k) == 4 : 
            tmp.append(k['word'])
            tmp.append(k['prefixe'])
            tmp.append(k['root'])
            tmp.append(k['suffixe'])
            L.append(tmp)
    return L
data = dic_to_list(final)

In [16]:
final_l = dic_to_list(final)
final_l[0]

['كَتِّبْ', '', 'كتب', '']

In [17]:
# experimenting with our first neural network to predict the root of a word.

data_1 = []

for word in final_l:
    tmp = []
    word[0] = word[0].replace('+', ' ')
    word[2] = word[2].replace('+', ' ')
    tmp.append(word[0])
    tmp.append(word[2])
    data_1.append(tmp)

data_1 = np.array(data_1)

In [18]:
# we extract our dictionnary from the our dataset 
def extract_dict(listt) :
    dictt = []
    for word in listt :
        for item in word : 
            tmp = set(item)
            for k in tmp : 
                if k not in dictt : 
                    dictt.append(k)
    return dictt       
dic = extract_dict(data_1)

### Let's create a data structure for each word of the corpus

In [ ]:
tmp = work_list[2]
tmp.split(":")

In [ ]:
vowel_pattern = r'[\u064b\u064e\u064f\u0650\u0651\u0652]'
word = "مَرْحَبًا"
vowels = re.findall(vowel_pattern, word)
print(f"Vowels in '{word}': {vowels}")

In [ ]:
'''filtered_split_list = []
for item in split_list : 
    temp_item_list = []
    for subitem in item : 
        if 'لا توجد نتائج لتحليل هذه الكلمة' not in item : 
            temp_item_list.append(item)
    filtered_split_list.append(temp_item_list)
filtered_split_list[0]'''

In [ ]:
'''# Check the data type of each element in texts
texts = pd.DataFrame(data_1[:, -1])
texts = texts.iloc[:, 1:]
for i, text in enumerate(texts):
    if not isinstance(text, str):
        print(f"Element {i} is not a string: {text}")'''